LOAD DATA

In [9]:
import pandas as pd
import numpy as np
df_raw=pd.read_csv("/kaggle/input/datasets/bayan16/day15-project/day15_real_dataset_large.csv")
print(df_raw.head())
print(df_raw.info())
print(df_raw.nunique().sort_values(ascending=False).head(10))

       age    income     city signup_time
0       30   55000.0       NY  2024-01-01
1      NaN   70000.0       SF  2024-01-05
2       45       NaN       LA  not a date
3  unknown  120000.0       NY  2024/02/01
4       28   65000.0  Chicago  2024-01-15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          78 non-null     object 
 1   income       77 non-null     float64
 2   city         96 non-null     object 
 3   signup_time  97 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB
None
signup_time    95
income         62
city           59
age            30
dtype: int64


PLAN

In [5]:
cleaning_plan = {
"age": {"type": "float", "missing": "median_imp", "outliers": "cap_99"},
"income": {"type": "float", "missing": "median_imp", "outliers": "log1p_cap_99"},
"city": {"type": "category", "clean": "canonical_city"},
"signup_time": {"type": "datetime", "tz": "UTC"}
}

FUNCTION

In [8]:
def capping(s:pd.Series,upper_cap:float=0.99)->pd.Series:
    upper=s.quantile(upper_cap)
    return s.clip(upper=upper)

def clean_ci(s:pd.Series)->pd.Series:
    canonical_map={"ny":"new york","nyc":"new york",
                   "sf":"san francisco","sanfrancisco":"san francisco",
                   "la":"los angeles"}
    cleaned=s.str.strip().str.lower()
    return cleaned.replace(canonical_map)

PROJECT CLEAN 

In [11]:
def clean_data_project(df_raw):
    df = df_raw.copy()
    df["age"]=pd.to_numeric(df["age"],errors="coerce")
    df["income"]=pd.to_numeric(df["income"],errors="coerce")
    df["signup_time"]=pd.to_datetime(df["signup_time"],errors="coerce")
    df.loc[df["age"]<=0,"age"]=np.nan
    df["age"]=df["age"].fillna(df["age"].median())
    df["income"]=df["income"].fillna(df["income"].median())

    df["income"]=capping(df["income"],upper_cap=0.99)
    df["city"]=clean_ci(df["city"])
    df=df.dropna(subset=["signup_time"]).copy()
    if df["signup_time"].dt.tz is None:
        df["signup_time"]=df["signup_time"].dt.tz_localize("UTC")
    return df
df_clean = clean_data_project(df_raw)

print("--- FINAL VERIFICATION ---")
print(f"Original rows: {len(df_raw)}")
print(f"Cleaned rows: {len(df_clean)}")
print("\n--- Summary Statistics ---")
print(df_clean[["age", "income"]].describe())
print(df_clean.info())
print(df_clean["city"].value_counts().head())
print("Timezone:",df_clean["signup_time"].dt.tz)
print(df_clean.isna().sum())

df_clean.to_csv("cleaned_dataset.csv", index=False)

--- FINAL VERIFICATION ---
Original rows: 97
Cleaned rows: 42

--- Summary Statistics ---
             age         income
count  42.000000      42.000000
mean   35.714286   76309.523810
std     5.701030   17339.491223
min    26.000000   45000.000000
25%    32.000000   64250.000000
50%    36.000000   75000.000000
75%    37.000000   87750.000000
max    49.000000  115000.000000
<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 0 to 94
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   age          42 non-null     float64            
 1   income       42 non-null     float64            
 2   city         41 non-null     object             
 3   signup_time  42 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), object(1)
memory usage: 1.6+ KB
None
city
new york         2
san francisco    2
chicago          2
austin           2
san diego        2
Name: count, dtype: 

ALL THE CODE

In [10]:
import pandas as pd
import numpy as np
df_raw=pd.read_csv("/kaggle/input/datasets/bayan16/day15-project/day15_real_dataset_large.csv")
print(df_raw.head())
print(df_raw.info())
print(df_raw.nunique().sort_values(ascending=False).head(10))

cleaning_plan = {
"age": {"type": "float", "missing": "median_imp", "outliers": "cap_99"},
"income": {"type": "float", "missing": "median_imp", "outliers": "log1p_cap_99"},
"city": {"type": "category", "clean": "canonical_city"},
"signup_time": {"type": "datetime", "tz": "UTC"}
}



def capping(s:pd.Series,upper_cap:float=0.99)->pd.Series:
    upper=s.quantile(upper_cap)
    return s.clip(upper=upper)

def clean_ci(s:pd.Series)->pd.Series:
    canonical_map={"ny":"new york","nyc":"new york",
                   "sf":"san francisco","sanfrancisco":"san francisco",
                   "la":"los angeles"}
    cleaned=s.str.strip().str.lower()
    return cleaned.replace(canonical_map)

def clean_data_project(df_raw):
    df = df_raw.copy()
    df["age"]=pd.to_numeric(df["age"],errors="coerce")
    df["income"]=pd.to_numeric(df["income"],errors="coerce")
    df["signup_time"]=pd.to_datetime(df["signup_time"],errors="coerce")
    df.loc[df["age"]<=0,"age"]=np.nan
    df["age"]=df["age"].fillna(df["age"].median())
    df["income"]=df["income"].fillna(df["income"].median())

    df["income"]=capping(df["income"],upper_cap=0.99)
    df["city"]=clean_ci(df["city"])
    df=df.dropna(subset=["signup_time"]).copy()
    if df["signup_time"].dt.tz is None:
        df["signup_time"]=df["signup_time"].dt.tz_localize("UTC")
    return df
df_clean = clean_data_project(df_raw)

print("--- FINAL VERIFICATION ---")
print(f"Original rows: {len(df_raw)}")
print(f"Cleaned rows: {len(df_clean)}")
print("\n--- Summary Statistics ---")
print(df_clean[["age", "income"]].describe())
print(df_clean.info())
print(df_clean["city"].value_counts().head())
print("Timezone:",df_clean["signup_time"].dt.tz)
print(df_clean.isna().sum())

df_clean.to_csv("cleaned_dataset.csv", index=False)

       age    income     city signup_time
0       30   55000.0       NY  2024-01-01
1      NaN   70000.0       SF  2024-01-05
2       45       NaN       LA  not a date
3  unknown  120000.0       NY  2024/02/01
4       28   65000.0  Chicago  2024-01-15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          78 non-null     object 
 1   income       77 non-null     float64
 2   city         96 non-null     object 
 3   signup_time  97 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB
None
signup_time    95
income         62
city           59
age            30
dtype: int64
--- FINAL VERIFICATION ---
Original rows: 97
Cleaned rows: 42

--- Summary Statistics ---
             age         income
count  42.000000      42.000000
mean   35.714286   76309.523810
std     5.701030   17339.491223
min    26.000000   45000.000000
25